In [2]:
%pip install --upgrade --quiet yandexcloud langchain_community langchain httpx


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


<a id='section-id3'></a>
### Настройте подключение к облаку

In [3]:
%pip uninstall jwt -q 
%pip install PyJWT -U -q


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
import json
import time
import jwt
import os
import httpx

Чтобы обратиться к API YandexGPT, вам потребуется сервисный аккаунт с ролью `ai.languageModels.user`.
1. Создайте сервисный аккаунт, как описано в [инструкции](https://cloud.yandex.ru/docs/iam/operations/sa/create).
2. В ячейке ниже укажите идентификатор сервисного аккаунта.
3. [Создайте](https://cloud.yandex.ru/docs/iam/operations/authorized-key/create) авторизованный ключ для сервисного аккаунта. 
4. Сохраните значение ключа в секрете `private-key`. [Как создать секрет](https://cloud.yandex.ru/docs/datasphere/operations/data/secrets).
5. Идентификатор ключа укажите в ячейке ниже.

In [5]:
service_account_id = os.environ['service_account_id']
key_id = os.environ['key_id']
private_key = os.environ["private-key"]

In [6]:
now = int(time.time())
payload = {
        'aud': 'https://iam.api.cloud.yandex.net/iam/v1/tokens',
        'iss': service_account_id,
        'iat': now,
        'exp': now + 360}

# Формирование JWT
encoded_token = jwt.encode(
    payload,
    private_key,
    algorithm='PS256',
    headers={'kid': key_id})

url = 'https://iam.api.cloud.yandex.net/iam/v1/tokens'
x = httpx.post(url,  headers={'Content-Type': 'application/json'}, json = {'jwt': encoded_token}).json()
token = x['iamToken']

In [7]:
catalog_id = os.environ["catalog-id"]

<a id='section-id2'></a>
## Подготовка данных и хранилища

In [58]:
import pandas as pd
from langchain_core.documents import Document

In [59]:
PART_PROCENT = 0.1;

In [60]:
dataset = []

template = """\
Наименование: {name}
Описание: {description}
"""


### Работа с мероприятиями

In [61]:
events = pd.read_json("data/events.json")
events = events[: int(events.shape[0] * PART_PROCENT)]
events

,title,description,min_age,price
0,Музей ВДНХ,"<p>Музей, посвященный истории ВДНХ. Обязателен...",NaN,NaN
1,Государственный музей А.С. Пушкина,<p>Двести лет назад званые вечера в ампирном о...,NaN,NaN
2,Макет космического корабля «Буран» на ВДНХ,<p>Макет многоразового орбитального ракетоплан...,NaN,NaN
3,«Москвариум» на ВДНХ,<p>Москвариум на ВДНХ является одним из крупне...,NaN,NaN
4,Выставка «Пленэр БКЛ»,<p>Несколько месяцев художники-пленэристы пров...,NaN,NaN
...,...,...,...,...
1079,Храм во имя Пресвятой Живоначальной Троицы,"<p>Храм был построен и освящен в 2006 году, на...",NaN,NaN
1080,Пляж «Челохово»,<p>В 2021 году пляж в Челохове вошел в список ...,NaN,NaN
1081,Преображенский собор,<p>Главный храм города построили после войны 1...,NaN,NaN
1082,Пустынские озера,<p>Уникальный природный комплекс из восьми озе...,NaN,NaN


In [62]:
for _, event in events.iterrows():
    description = event["description"].replace('<p>', '').replace('</p>', '') 
    document = Document(page_content=template.format(name=event["title"], description=description), metadata={"type": "events"})
    dataset.append(document)

In [63]:
dataset[: 10]

[Document(metadata={'type': 'events'}, page_content='Наименование: Музей ВДНХ\nОписание: Музей, посвященный истории ВДНХ. Обязателен для посещения любителям советской эстетики и программы «Намедни». Находится в правом крыле арки главного входа. Внутри вас ждет большой архив с редкими фотографиями, чертежами павильонов и погружение в советскую историю.\n'),
 Document(metadata={'type': 'events'}, page_content='Наименование: Государственный музей А.С. Пушкина\nОписание: Двести лет назад званые вечера в ампирном особняке собирали весь высший свет Москвы. Почувствуйте себя знатной особой, перед которой гостеприимно распахнутся двери усадьбы Хрущевых. Перед вами оживут Пушкин и его современники, их непринужденные беседы и горячие дискуссии о судьбах Родины. В залах, посвященных биографии поэта, вы увидите свидетельства становления гения. А юные экскурсанты смогут отправиться на остров Буян, побывать на ярмарке и в других местах волшебного мира сказок Пушкина.\n'),
 Document(metadata={'type':

### Работа с местами

In [64]:
places = pd.read_json("data/places.json")
places

,title,description
0,Воронежская область,Воронежская область — сердце Черноземья и край...
1,Кемеровская область,Кузбасс – регион с невероятной природой и фант...
2,Карачаево-Черкесская Республика,"Древняя земля, воспетая поэтами, одна из горно..."
3,Курганская область,"Край провинциальной старины, в котором оставил..."
4,Смоленская область,Живописный край в верховьях Днепра с крепостны...
...,...,...
177,Ненецкий автономный округ,Ненецкий автономный округ – удивительный и заг...
178,Орловская область,Уголок настоящей русской природы в четырех час...
179,Сахалинская область,"Уникальный регион России, полностью расположен..."
180,Краснодарский край,"Летом вас ждут пляжи Сочи, Анапы и Геленджика,..."


In [65]:
for _, place in places.iterrows():
    description = place["description"].replace('<p>', '').replace('</p>', '') 
    document = Document(page_content=template.format(name=place["title"], description=description), metadata={"type": "places"})
    dataset.append(document)
    

In [66]:
dataset[-10: ]

[Document(metadata={'type': 'places'}, page_content='Наименование: Иркутская область\nОписание: Одно из тех мест, которые природа создала с особой любовью. Вас ждет захватывающее путешествие по загадочным островам, древним пещерам и знакомство с легендарным озером Байкал.\n'),
 Document(metadata={'type': 'places'}, page_content='Наименование: Кабардино-Балкарская Республика\nОписание: Регион с многовековой историей, уникальной культурой и потрясающей природой. Именно здесь находится самая высокая точка России – величественная двуглавая гора Эльбрус.\n'),
 Document(metadata={'type': 'places'}, page_content='Наименование: Тверская область\nОписание: Обаятельный край с невероятной природой, где старинные легенды о героях и святых органично соседствуют с современностью.\n'),
 Document(metadata={'type': 'places'}, page_content='Наименование: Новгородская область\nОписание: Новгородская область — «колыбель России» и ее государственности, исторический центр распространения православия и земля

### Работа с ресторанами

In [67]:
restaurant_template = """\
Наименование: {name}
Описание: {description}
Адрес: {address}
Город: {city}
Кухня: {cuisine}
"""

In [68]:
restoraunts = pd.read_json("data/restoraunts.json")
restoraunts = restoraunts[:int(restoraunts.shape[0] * PART_PROCENT)]
restoraunts

,title,description,address,cities,cuisines
0,«Московское чаепитие» в бутик-отеле «Петровски...,<p><strong>Отель участвует в проекте «Московск...,"Ленинградский пр-т, д. 40, г. Москва",[Москва],[Европейская]
1,Ресторан «Мясо & Рыба» на Пресненской,<p>Гармоничное сочетание стейк-хауса и рыбного...,"Пресненская наб., д. 2, г. Москва",[Москва],"[Европейская, Рыбная, Мясная]"
2,Ресторан Björn,<p>Первый zero waste ресторан в России. Сезонн...,"ул. Пятницкая, д. 3/4, стр. 1, г. Москва",[Москва],[Северная]
3,Ресторан Hedonist,<p>Ресторан современной грузинской кухни на По...,"Покровский бул., д. 8, стр. 1, г. Москва",[Москва],[Грузинская]
4,Ресторан «Турандот»,<p>Ресторан «Турандот» открылся в 2005 году. У...,"Тверской бул., д. 26, г. Москва",[Москва],"[Европейская, Японская, Паназиатская, Китайская]"
...,...,...,...,...,...
1369,Ресторан «Папа Карло»,<p>Светлый и просторный ресторан итальянской к...,"ул. Говорова, д. 18, г. Москва",[Москва],"[Итальянская, Европейская, Кавказская]"
1370,Пиццерия Mates Pizza & Bar в Хамовниках,"<p>Двухэтажный бар, основное меню которого сос...","ул. Усачева, д. 11А, г. Москва",[Москва],"[Итальянская, Европейская]"
1371,"Ресторан One Special, Павелецкая площадь",<p>Концепция проектов One Special стоит на тре...,"Павелецкая пл., д. 3, г. Москва (ТРЦ «Павелецк...",[Москва],[Европейская]
1372,Кафе «Оттепель»,<p>Кафе расположено в спортивном комплексе «Сн...,"дер. Марушкино, д. 2, г. Москва",[Москва],"[Русская, Итальянская, Европейская, Американская]"


In [69]:
for _, restaurant in restoraunts.iterrows():
    description = (restaurant["description"].replace('<p>', '')
                                            .replace('</p>', '')
                                            .replace('<strong>', '')
                                            .replace('</strong>', ''))
    city = "".join(restaurant['cities'])
    cuisines = ",".join(restaurant['cuisines'])
    document = Document(page_content=restaurant_template.format(name=restaurant['title'], description=description, address=restaurant['address'],
                                                                city=city,cuisine=cuisines), metadata={"type": "restaurants"})
    dataset.append(document)
    

In [70]:
dataset[-5:]

[Document(metadata={'type': 'restaurants'}, page_content='Наименование: Ресторан «Папа Карло»\nОписание: Светлый и просторный ресторан итальянской кухни. Здесь можно сытно и вкусно поужинать после трудового дня, отметить день рождения или организовать банкет в роскошном зале. В «Папе Карло» подают легкие утренние закуски-брускетты с различными начинками, готовят несколько видов пицц, пасту с мясом, овощами и морепродуктами, а также жарят ребрышки с прованскими травами и предлагают нежные десерты. В барной карте представлен достойный выбор выдержанных вин и других напитков. В теплое время года открывается летняя веранда. \nАдрес: ул. Говорова, д. 18, г. Москва\nГород: Москва\nКухня: Итальянская,Европейская,Кавказская\n'),
 Document(metadata={'type': 'restaurants'}, page_content='Наименование: Пиццерия Mates Pizza & Bar в Хамовниках\nОписание: Двухэтажный бар, основное меню которого состоит из пиццы и авторских сэндвичей. Согласно неаполитанским традициям блюда здесь готовятся в дровяной

### Работа с направлением путешествий

In [71]:
track_infos = pd.read_json("data/track_info.json")
track_infos

,title,description,cities,event_types,type_travel
0,Москва в деталях: прогулка по Парку Горького,<p>Одно из знаковых мест столицы ― Парк Горько...,[Москва],"[Парк культуры и отдыха, Музей, Набережная, Ал...",[Пеший]
1,Большая прогулка в «Сокольниках»,<p>В XV столетии на территории современного па...,[Москва],"[Аллея, Пруды, Парк культуры и отдыха, Архитек...",[Пеший]
2,Яуза и окрестности: прогулка по Москве для все...,<p>Не каждый мегаполис может похвастаться множ...,[Москва],"[Архитектура, Парк культуры и отдыха, Храмы и ...",[Смешанный]
3,Государев двор и окрестности: прогулка по Коло...,<p>Расположенное на юге Москвы Коломенское был...,[Москва],"[Смотровая площадка , Парк культуры и отдыха, ...",[Пеший]
4,От народного творчества к современному искусству,<p>Русское народное творчество не перестает во...,[Москва],"[Галерея , Музей, Театр, Цирк , Парк развлечен...",[Смешанный]
...,...,...,...,...,...
517,Незабываемый день в Ярославле,<p>Поездка в Ярославль — отличный вариант для ...,[Ярославль],[],[Смешанный]
518,На пути к «Матери-Волге»: знакомство с Рыбинском,<p>Верхневолжский город знаменит большим Рыбин...,[Рыбинск],[Достопримечательности],[Смешанный]
519,В поисках Челябинского метеорита,"<p>Один метеорит, путешествуя по Вселенной и п...",[Челябинск],"[Парк культуры и отдыха, Музей, Достопримечате...",[Пеший]
520,"Ярославская старина, ивановский авангард: путе...",<p>Уникальные образцы древнерусского зодчества...,[Ярославль],"[Музей, Достопримечательности, Театр]",[Смешанный]


In [72]:
track_info_template = """\
Наименование: {name}
Описание: {description}
Города: {cities}
Типы мест: {event_types}
Тип путешествия: {type}
"""

In [73]:
for _, track_info in track_infos.iterrows():
    description = (track_info["description"].replace('<p>', '')
                                            .replace('</p>', '')
                                            .replace('<strong>', '')
                                            .replace('</strong>', ''))
    cities = ",".join(track_info["cities"])
    event_types = ",".join(track_info["event_types"])
    type_travel = ",".join(track_info["type_travel"])
    document = Document(page_content=track_info_template.format(name=track_info["title"], description=description, cities=cities, event_types=event_types, type=type_travel), metadata={"type": "tracks"})
    dataset.append(document)
    

In [74]:
dataset[-10:]

[Document(metadata={'type': 'tracks'}, page_content='Наименование: Севастопольский вальс\nОписание: Маршрут «Севастопольский вальс» посвящен героической 250-дневной обороне города в годы Великой Отечественной войны. Вы посетите казематы 35-й береговой батареи, побываете в колоритном здании Музея Черноморского флота, а в конце подниметесь на легендарную Сапун-гору, где до сих пор сохранились следы окопов и траншей.\nГорода: Севастополь\nТипы мест: Музей,Достопримечательности\nТип путешествия: Пеший\n'),
 Document(metadata={'type': 'tracks'}, page_content='Наименование: Два незабываемых дня в Туле\nОписание: Тула — старинный город, который прославился своими ремесленниками и мастерами оружейного дела. На поездку в это поистине сказочное место стоит потратить целые выходные. За два дня вы успеете погрузиться в местную историю, побывать на дегустации тульского пряника, оказаться в атмосфере романов великого Льва Толстого и многое другое!\nГорода: Тула\nТипы мест: Музей,Арт\nТип путешествия

### Подготовка векторизации

In [56]:
%pip install faiss-gpu

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [9]:
import faiss
import statistics
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import YandexGPTEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore


In [93]:
lengths = [len(data.page_content) for data in dataset]
m = statistics.mean(lengths)
sd = statistics.stdev(lengths)


print(f"Всего документов базе: {len(dataset)}") 
print("Mean %d" % m)
print("SD %d" % sd)

potential_length = m + 1.5 * sd

Всего документов базе: 4246
Mean 577
SD 206


In [12]:
embeddings = YandexGPTEmbeddings(iam_token=token, folder_id=catalog_id, sleep_interval=0.1)

# storage = FAISS.from_documents(dataset, embedding=embeddings)

In [96]:
# storage.save_local("travel_store")

In [13]:
storage = FAISS.load_local(
    "travel_store", embeddings, allow_dangerous_deserialization=True
)

<a id='section-id2'></a>
## RAG

<a id='section-id3'></a>
### Простой RAG

In [15]:
from langchain_community.llms import YandexGPT
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.chains import HypotheticalDocumentEmbedder

/home/jupyter/.local/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_uri" in _BaseYandexGPT has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/jupyter/.local/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _BaseYandexGPT has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/jupyter/.local/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_version" in _BaseYandexGPT has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [17]:
model_version = "yandexgpt/rc"
model = YandexGPT(modelUri=f"gpt://{catalog_id}/{model_version}", iam_token=token, folder_id=catalog_id)
simple_retriever = storage.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "fetch_k": 6, "lambda_mult": 0.5 }
)
rag_template = """\
Ты выступаешь в роли туристического консультанта. Твоя задача — помогать пользователям, предоставляя точную, полезную и интересную информацию о достопримечательностях, культурных особенностях, истории и традициях различных стран и городов мира. Твоя цель — помочь людям планировать поездки, делая их более увлекательными и познавательными.

Вот твои основные обязанности:

Предоставление информации о достопримечательностях: Описание музеев, архитектурных памятников, природных объектов, культурных событий и популярных мест для туристов.
Исторический контекст: Объяснение исторических событий, которые связаны с конкретными местами или регионами, объяснение значимости этих мест в контексте мировой истории.
Рекомендации по маршрутам: Помощь в составлении маршрутов по городам или регионам, включая советы по времени посещения, транспорту, а также о том, что стоит увидеть в первую очередь.
Информация о культуре и традициях: Объяснение местных обычаев, традиций и особенностей поведения в различных странах.
Ответы на специфические запросы: Предоставление информации по интересующим вопросам, таким как местные кухни, лучшие рестораны, советы по безопасности, сезонные особенности и климат.
Ты обязан придерживаться следующих принципов:

Давай точную, проверенную информацию.
Ответ должен соедржать подробную информацию, почему человек должен посетить указанные места.
Помогай создавать познавательные маршруты, добавляя историческую ценность в каждую рекомендацию.
Не забывай быть дружелюбным, внимательным и готовым к любым вопросам от пользователей!


Используйте следующий контекст, чтобы помочь ответить на вопрос. 

{context}

{question}
"""



prompt = PromptTemplate.from_template(rag_template)

chain = (
    { "context": simple_retriever, "question" : RunnablePassthrough() }
    | prompt
    | model
    | StrOutputParser()
)



In [51]:
print(chain.invoke("Назови пять мест обязательных для посещения, если хочется попробовать о китайскую еду"))

1. Ресторан «Китайская грамота» на Сретенке — здесь можно попробовать эксперименты шеф-повара в рамках правильной техники приготовления китайских блюд.
2. Чихо — комьюнити творческих ребят, которые погружают гостей в атмосферу китайской улицы и предлагают попробовать китайский стрит-фуд.
3. Ресторан China Today — предлагает своим гостям насладиться аутентичными блюдами Азии в современной интерпретации.
4. Ресторан «Имбирь» — возможно, стоит посетить его для сравнения с другими заведениями, предлагающими китайскую кухню.
5. Brasserie Мост — переделанный из ресторана The Мост, он также может предложить интересные варианты блюд китайской кухни.


In [54]:
print(chain.invoke("Назови пять мест обязательных для посещения, если хочется узнать историю собак в космосе"))

1. Музей собаки — здесь представлены документы, вещи, медали, значки, ошейники, книги, фотографии, открытки, картины и рисунки, свидетельствующие о различных периодах российской истории, когда жизни человека и собаки были неотделимы друг от друга.
2. Маршрут «Космическая одиссея» в Москве — позволит погрузиться в атмосферу 60-х годов прошлого века, времени первого полёта человека в космос.
3. Аллея Героев Космоса — место, где установлены памятники и бюсты выдающихся космонавтов.
4. Мемориальный музей космонавтики — здесь можно узнать об истории освоения космоса, увидеть настоящие космические аппараты и оборудование.
5. Музей Московского космического кластера — интерактивная экспозиция, посвящённая истории и достижениям отечественной космонавтики.


<a id='section-id3'></a>
### HyDE

In [79]:
hyde_prompt_template="""\
Ты выступаешь в роли туристического консультанта. Твоя задача — помогать пользователям, предоставляя точную, полезную и интересную информацию о достопримечательностях, культурных особенностях, истории и традициях различных стран и городов мира. Твоя цель — помочь людям планировать поездки, делая их более увлекательными и познавательными.

Вот твои основные обязанности:

Предоставление информации о достопримечательностях: Описание музеев, архитектурных памятников, природных объектов, культурных событий и популярных мест для туристов.
Исторический контекст: Объяснение исторических событий, которые связаны с конкретными местами или регионами, объяснение значимости этих мест в контексте мировой истории.
Рекомендации по маршрутам: Помощь в составлении маршрутов по городам или регионам, включая советы по времени посещения, транспорту, а также о том, что стоит увидеть в первую очередь.
Информация о культуре и традициях: Объяснение местных обычаев, традиций и особенностей поведения в различных странах.
Ответы на специфические запросы: Предоставление информации по интересующим вопросам, таким как местные кухни, лучшие рестораны, советы по безопасности, сезонные особенности и климат.
Ты обязан придерживаться следующих принципов:

Давай точную, проверенную информацию.
Уважай культурные особенности и избегай стереотипов.
Давай советы, которые могут быть полезны для различных типов туристов, включая тех, кто путешествует с семьей, одиноко или в группе.
Помогай создавать познавательные маршруты, добавляя историческую ценность в каждую рекомендацию.

Не забывай быть дружелюбным, внимательным и готовым к любым вопросам от пользователей!

Question: {question}
Answer:"""

prompt = PromptTemplate(input_variables=["question"], template=hyde_prompt_template)
hyde_model = YandexGPT(modelUri=f"gpt://{catalog_id}/{model_version}", iam_token=token, folder_id=catalog_id)
llm_chain = prompt | hyde_model
hyde_embeddings = HypotheticalDocumentEmbedder(llm_chain=llm_chain, base_embeddings=embeddings)
hyde_storage = FAISS.from_documents(dataset, embedding=embeddings)

In [80]:
hyde_retriever = hyde_storage.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "fetch_k": 6, "lambda_mult": 0.5 }
)

prompt = PromptTemplate.from_template(rag_template)

hyde_chain = (
    { "context": hyde_retriever, "question" : RunnablePassthrough() }
    | prompt
    | model
    | StrOutputParser()
)



In [82]:
print(hyde_chain.invoke("Назови пять мест обязательных для посещения, если хочется попробовать о китайскую еду"))

1. Ресторан «Китайская грамота» на Сретенке. Этот ресторан открыл известный адвокат и любитель гастрономии Александр Раппопорт. В меню есть не только воки, димсамы и битые огурцы, но и более экзотичные блюда, например, свиные уши, говяжьи языки, куриные лапки и телячьи желудки.
2. Чихо. Не просто китайская закусочная, а комьюнити творческих ребят, которые погружают гостей в атмосферу китайской улицы. Здесь можно найти китайский стрит-фуд: лапшу-вок, рис, закуски и десерты.
3. Ресторан China Today. Ресторан китайской и паназиатской кухни предлагает своим гостям насладиться аутентичными блюдами Азии в современной интерпретации. В меню есть как классические блюда, так и более экзотичные, например, акульи плавники, жареные голуби, ласточкины гнёзда и глубоководный моллюск.
4. «Имбирь». Один из первых ресторанов, открытых Александром Раппопортом. В меню есть разнообразные блюда китайской кухни.
5. «The Мост» (сейчас «Brasserie Мост»). После реконструкции здесь можно попробовать блюда китайс

In [84]:
print(hyde_chain.invoke("Назови пять мест обязательных для посещения, если хочется узнать историю собак в космосе"))

1. **Музей собаки.** Этот музей является хранителем многовековой дружбы человека и собаки. В экспозиции представлены документы, вещи, медали, значки, ошейники, книги, фотографии, открытки, картины и рисунки, свидетельствующие о различных периодах российской истории, когда жизни человека и собаки были неотделимы друг от друга.

2. **Космическая одиссея.** Маршрут «Космическая одиссея» позволит вам погрузиться в атмосферу 60-х годов прошлого века — времени первого полёта человека в космос.

3. **Парк покорителей космоса им. Юрия Гагарина.** Парк находится в месте, где Юрий Гагарин спустился на землю после своего легендарного полёта в космос. Здесь расположены уникальные объекты — скульптуры, обзорная площадка, мемориал в виде стены с инфографикой и копия автобуса, подвозившего астронавтов ко взлётному полю.

4. **Мемориальный музей космонавтики.** Музей, посвящённый истории освоения космоса, также может быть интересен для тех, кто хочет узнать больше о роли собак в космических исследован

<a id='section-id3'></a>
### Contextual Compression

In [93]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor, LLMChainFilter

In [96]:
compressor = LLMChainFilter.from_llm(model)
retriever = storage.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 10, "lambda_mult": 0.5 }
)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

contextual_chain = (
    { "context": compression_retriever , "question" : RunnablePassthrough() }
    | prompt
    | model
    | StrOutputParser()
)

In [97]:
print(contextual_chain.invoke("Назови пять мест обязательных для посещения, если хочется попробовать о китайскую еду"))

1. Ресторан «Китайская грамота» на Сретенке — здесь можно попробовать эксперименты в рамках правильной техники, которой обладает шеф, привезённый из Китая.
2. Чихо — комьюнити творческих ребят, которые погружают гостей в атмосферу китайской улицы. В Чихо можно найти китайский стрит-фуд: лапшу-вок, рис, закуски и десерты, все это готовят этнические китайцы.
3. Ресторан «Новая Азия» — здесь готовят национальные блюда Кореи и гастрономические хиты других стран Азии — Китая, Вьетнама, Таиланда.
4. Чайная «ЧаЕ» — уютное кафе, где посреди парка «Сокольники» можно насладиться качественным китайским чаем. Здесь проводят настоящие чайные церемонии, соблюдая все правила древних традиций.
5. Ресторан с корейской кухней «Новая Азия». Одна из особенностей ресторана — зал корейского барбекю, где гости лично или с помощью официантов поджаривают на специальной жаровне, встроенной в стол, различные виды мяса, морепродукты и овощи.


In [98]:
print(contextual_chain.invoke("Назови пять мест обязательных для посещения, если хочется узнать историю собак в космосе"))

1. **Музей собаки** — здесь представлены документы, вещи, медали, значки, ошейники, книги, фотографии, открытки, картины и рисунки, свидетельствующие о различных периодах российской истории, когда жизни человека и собаки были неотделимы друг от друга.
2. **Государственный музей истории космонавтики имени К. Э. Циолковского** — первый в мире музей космонавтики. В нём собрана одна из лучших и наиболее полных коллекций отечественной космической техники, включающая в себя аппараты, побывавшие в космосе.
3. **Павильон «Космос» на ВДНХ** — любимое место Юрия Гагарина, связанное с освоением космоса.
4. **Маршрут «По Москве на четырёх лапах»** — прогулка по местам, связанным с собаками и их знаменитыми хозяевами: Михаилом Булгаковым, Юрием Никулиным, Антоном Чеховым, Фёдором Шаляпиным.
5. **Мавзолей В. И. Ленина на Красной площади** — хотя напрямую не связан с историей собак в космосе, является важной частью исторического и культурного наследия России.


<a id='section-id3'></a>
### MultiQuery Retriever

In [21]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.output_parsers import BaseOutputParser

In [23]:
class LineListOutputParser(BaseOutputParser[list[str]]):

    def parse(self, text: str) -> list[str]:

        return list(filter(None, text.strip().split("\n")))

In [29]:
# MultiQuery prompt: different perpectives
from langchain.load import dumps, loads
from operator import itemgetter

mq_prompt_template = """\
Ты являешься помощником на основе языковой модели ИИ. 
Твоя задача — сформулировать пять различных версий заданного вопроса пользователя, чтобы получить релевантные документы из векторной базы данных.
Генерируя различные подходы к вопросу пользователя, твоя цель — помочь пользователю преодолеть некоторые ограничения поиска по сходству на основе расстояния.
Предоставь альтернативные вопросы, разделённые новой строкой.
Оригинальный вопрос: {question}
"""
prompt_perspectives = PromptTemplate.from_template(mq_prompt_template)

mq_model = prompt_perspectives | model | LineListOutputParser()

retriever = MultiQueryRetriever(

    retriever=storage.as_retriever(), llm_chain=mq_model, parser_key="lines", include_original=True

)


In [30]:
mq_chain = (
    { "context": retriever, "question" : RunnablePassthrough() }
    | prompt
    | model
    | StrOutputParser()
)

In [37]:
print(mq_chain.invoke("Назови пять мест для посещения , если хочется попробовать о китайскую еду"))

1. Ресторан «Китайские Новости» на Арбате — здесь можно попробовать фирменные пельмени сяолунбао, а также другие блюда юга Китая.
2. Ресторан China Today — предлагает аутентичные блюда Азии в современной интерпретации.
3. Ресторан «Китайская грамота» на Сретенке — первый собственный ресторан известного адвоката и любителя гастрономии Александра Раппопорта.
4. Ресторан «Новая Азия» — здесь готовят национальные блюда Кореи и гастрономические хиты других стран Азии — Китая, Вьетнама, Таиланда.
5. Чихо — комьюнити творческих ребят, которые погружают гостей в атмосферу китайской улицы. В Чихо можно найти китайский стрит-фуд: лапшу-вок, рис, закуски и десерты.


In [39]:
print(mq_chain.invoke("Назови пять мест обязательных для посещения в Краснодаре"))

1. **Музей-заповедник «Танаис»**. В устье Дона некогда стоял боспорский город, названный по тогдашнему имени реки — Танаис. Теперь на его месте огромный музей-заповедник: здесь по-прежнему ведут раскопки, а посетители могут познакомиться с его историей и даже увидеть крупнейшее собрание амфорных эталонов.
2. **Сочинский Дендрарий**. Прогулка по этому парку сродни путешествию по странам и континентам. На территории в 46 га собраны растения со всех концов света, в общей сложности 1800 видов. Тут и пальмы, и бамбук, и сосны. Радует пышным цветением розарий, а в прудах плавают лебеди и пеликаны.
3. **Парк культуры и отдыха в Краснодаре**. Южная красота города заставляет биться сердце чаще. Прогуляйтесь по улицам столицы Кубани, спуститесь к набережной и отдохните в живописном парке.
4. **Государственный природный заповедник «Ростовский»**. Бескрайние чернозёмные степи, наполненные ароматами дикого разнотравья, озёра и лиманы — природа «Ростовского» завораживает. Маршруты местных экологичес